In [1]:
pip install elasticsearch

You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

import os
import json
import pandas as pd
from elasticsearch import Elasticsearch

### **Spark**

#### Inicialización y lectura de archivos JSON con PySpark

In [4]:
spark = SparkSession.builder.master("local[*]").appName("practica_ibd").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 15:56:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
archivos_json = spark.read.option("multiline", "true").json("/opt/workspace")

#### Procesamiento y selección de campos

In [6]:
campos_seleccionados = archivos_json.select(
    archivos_json['title'].alias('title'),
    archivos_json['author_name'].getItem(0).alias('author_name'),
    archivos_json['first_publish_year'].alias('publish_year'),
    archivos_json['language'].alias('language'),
    archivos_json['subject'].alias('subject'),
    archivos_json['number_of_pages_median'].alias('num_pages'),
    archivos_json['id_amazon'].alias('id_amazon'),
    archivos_json['ratings_average'].alias('ratings')
)

print('Hay', campos_seleccionados.count(), 'archivos en el Dataframe.')

Hay 2241 archivos en el Dataframe.


#### Conversión a Pandas y limpieza de datos

In [7]:
campos_pandas = campos_seleccionados.toPandas()
campos_pandas.head()

,title,author_name,publish_year,language,subject,num_pages,id_amazon,ratings
0,None,None,NaN,None,None,NaN,None,NaN
1,Frankenstein or The Modern Prometheus,Mary Wollstonecraft Shelley,1818.0,"[fre, ita, por, ger, chi, rus, spa, eng, dut]","[Frankenstein (Fictitious character), Frankens...",240.0,"[B08M25T3PW, B093XMVLXF, B000LQ3XU2, 198604261...",3.900585
2,A Christmas Carol,Charles Dickens,1843.0,"[chi, spa, rus, eng, kor, fre, und, ita, fin, ...","[Ghost stories, Readers, Ebenzer Scrooge (Fict...",114.0,"[B002D6DY1G, B000QNQ8J8, , B01N40IX3W, B0022WF...",3.883117
3,The Picture of Dorian Gray,Oscar Wilde,1890.0,"[dut, chi, heb, arm, ukr, spa, fre, rus, cze, ...",[British and irish fiction (fictional works by...,253.0,"[B000OKWFA4, B008MWZR1A, B00ABMJJSQ, B00XF4RG4...",4.041666
4,Kalender 2021,Paula Paulsen,2020.0,[ger],None,120.0,None,NaN


In [8]:
filas_con_nulos = campos_pandas[campos_pandas.isnull().all(axis=1)]
indices_eliminar = filas_con_nulos.index.tolist()

campos_pandas.drop(indices_eliminar, inplace=True)

#### Guardado del Dataframe en el volumen

In [9]:
ruta_archivo_csv = '/opt/workspace/campos_pandas.csv'
campos_pandas.to_csv(ruta_archivo_csv, index=False)

In [10]:
spark.stop()

### **Elasticsearch**

#### Inserción de datos en Elasticsearch

In [11]:
client = Elasticsearch("http://elasticsearch:9200")

In [12]:
ruta_archivo_csv = '/opt/workspace/campos_pandas.csv'
campos_pandas = pd.read_csv(ruta_archivo_csv)
campos_pandas.head()

,title,author_name,publish_year,language,subject,num_pages,id_amazon,ratings
0,Frankenstein or The Modern Prometheus,Mary Wollstonecraft Shelley,1818.0,"['fre', 'ita', 'por', 'ger', 'chi', 'rus', 'sp...","['Frankenstein (Fictitious character)', ""Frank...",240.0,"['B08M25T3PW', 'B093XMVLXF', 'B000LQ3XU2', '19...",3.900585
1,A Christmas Carol,Charles Dickens,1843.0,"['chi', 'spa', 'rus', 'eng', 'kor', 'fre', 'un...","['Ghost stories', 'Readers', 'Ebenzer Scrooge ...",114.0,"['B002D6DY1G', 'B000QNQ8J8', '', 'B01N40IX3W',...",3.883117
2,The Picture of Dorian Gray,Oscar Wilde,1890.0,"['dut', 'chi', 'heb', 'arm', 'ukr', 'spa', 'fr...",['British and irish fiction (fictional works b...,253.0,"['B000OKWFA4', 'B008MWZR1A', 'B00ABMJJSQ', 'B0...",4.041666
3,Kalender 2021,Paula Paulsen,2020.0,['ger'],NaN,120.0,NaN,NaN
4,Northanger Abbey,Jane Austen,1818.0,"['und', 'eng', 'spa', 'por', 'cat', 'ger', 'ru...","['Mate selection', 'Cousins', 'Fiction', 'Youn...",254.0,"['B000H8TULK', 'B0012BZNYE', 'B000HMBM7Q', 'B0...",3.785714


In [13]:
for index, row in campos_pandas.iterrows():
    json_obj = row.to_json()
    
    client.index(index='my_index', body=json_obj)

#### Consultas y operaciones en Elasticsearch

##### Consulta básica

In [14]:
res = client.search(index='my_index', size=3)  

for hit in res['hits']['hits']:
    print(hit['_source'])

{'title': 'Frankenstein or The Modern Prometheus', 'author_name': 'Mary Wollstonecraft Shelley', 'publish_year': 1818.0, 'language': "['fre', 'ita', 'por', 'ger', 'chi', 'rus', 'spa', 'eng', 'dut']", 'subject': '[\'Frankenstein (Fictitious character)\', "Frankenstein\'s monster (Fictitious character)", \'Fiction\', \'Victor Frankenstein (Fictitious character)\', \'Scientists\', \'Monsters\', \'Fiction, horror\', \'Frankenstein (fictitious character), fiction\', \'Physicians, fiction\', \'British fiction (fictional works by one author)\', \'Scientists, fiction\', \'Geneva (Switzerland), fiction\', "Children\'s fiction", \'Horror stories\', \'Fiction, science fiction, general\', \'Fiction, general\', \'Horror tales\', \'Frankenstein (Shelley, Mary Wollstonecraft)\', \'Large type books\', \'English fiction\', \'Study guides\', \'Examinations\', \'Criticism and interpretation\', \'Fiction, gothic\', \'English language, textbooks for foreign speakers\', \'Reading comprehension\', \'Monsters

##### Conteo de documentos

In [15]:
doc_count = client.count(index='my_index')['count']
print(f"El número total de documentos en el índice es: {doc_count}")

El número total de documentos en el índice es: 2239


##### Obtener IDs de documentos

In [16]:
query = {
    "_source": False,  
    "query": {
        "match_all": {}  
    }
}

res = client.search(index='my_index', body=query)

document_ids = [hit['_id'] for hit in res['hits']['hits']]
print("IDs de los documentos en el índice:")
for doc_id in document_ids:
    print(f"  - {doc_id}")

IDs de los documentos en el índice:
  - IQ0IoY8BH564STdAkrxw
  - Ig0IoY8BH564STdAkryk
  - Iw0IoY8BH564STdAkryq
  - JA0IoY8BH564STdAkryw
  - JQ0IoY8BH564STdAkry1
  - Jg0IoY8BH564STdAkry8
  - Jw0IoY8BH564STdAkrzB
  - KA0IoY8BH564STdAkrzH
  - KQ0IoY8BH564STdAkrzN
  - Kg0IoY8BH564STdAkrzT


##### Obtener campos únicos de un documento específico

In [17]:
doc_id = 'IQ0IoY8BH564STdAkrxw'  
res = client.get(index='my_index', id=doc_id)

print(res['_source'].keys())

dict_keys(['title', 'author_name', 'publish_year', 'language', 'subject', 'num_pages', 'id_amazon', 'ratings'])


##### Obtener un documento por su ID

In [18]:
doc_id = 'IQ0IoY8BH564STdAkrxw'  

try:
    res = client.get(index='my_index', id=doc_id)
    
    print(f"Campos y valores del documento con ID '{doc_id}':")   
    for field, value in res['_source'].items():
        print(f"  - {field}: {value}")

except Exception as e:
    print(f"No se pudo obtener el documento con ID '{doc_id}'. Error: {e}")

Campos y valores del documento con ID 'IQ0IoY8BH564STdAkrxw':
  - title: Frankenstein or The Modern Prometheus
  - author_name: Mary Wollstonecraft Shelley
  - publish_year: 1818.0
  - language: ['fre', 'ita', 'por', 'ger', 'chi', 'rus', 'spa', 'eng', 'dut']
  - subject: ['Frankenstein (Fictitious character)', "Frankenstein's monster (Fictitious character)", 'Fiction', 'Victor Frankenstein (Fictitious character)', 'Scientists', 'Monsters', 'Fiction, horror', 'Frankenstein (fictitious character), fiction', 'Physicians, fiction', 'British fiction (fictional works by one author)', 'Scientists, fiction', 'Geneva (Switzerland), fiction', "Children's fiction", 'Horror stories', 'Fiction, science fiction, general', 'Fiction, general', 'Horror tales', 'Frankenstein (Shelley, Mary Wollstonecraft)', 'Large type books', 'English fiction', 'Study guides', 'Examinations', 'Criticism and interpretation', 'Fiction, gothic', 'English language, textbooks for foreign speakers', 'Reading comprehension', 

##### Búsqueda por rango de ratings

In [19]:
gte_rating = 4.0
lte_rating = 5.0

query = {
    "query": {
        "range": {
            "ratings": {
                "gte": gte_rating,
                "lte": lte_rating
            }
        }
    }
}

res = client.search(index='my_index', body=query)

print(f"Documentos con ratings entre {gte_rating} y {lte_rating}:")
for hit in res['hits']['hits']:
    source = hit['_source']
    titulo = source.get('title', 'Título desconocido')
    autor = source.get('author_name', 'Autor desconocido') 
    ratings = source.get('ratings', 'Ratings desconocidos')
    print(f"  - Título: {titulo}, Autor: {autor}, Ratings: {ratings}")

Documentos con ratings entre 4.0 y 5.0:
  - Título: The Picture of Dorian Gray, Autor: Oscar Wilde, Ratings: 4.0416665
  - Título: It, Autor: Stephen King, Ratings: 4.109005
  - Título: Works (Carrie / Night Shift / 'Salem's Lot / Shining), Autor: Stephen King, Ratings: 4.5555553
  - Título: The Stand, Autor: Stephen King, Ratings: 4.285714
  - Título: Skeleton Crew, Autor: Stephen King, Ratings: 4.347826
  - Título: Different Seasons, Autor: Stephen King, Ratings: 4.5882354
  - Título: Salem’s Lot, Autor: Stephen King, Ratings: 4.0882354
  - Título: The King in Yellow, Autor: Robert W. Chambers, Ratings: 4.142857
  - Título: Poems, Autor: Percy Bysshe Shelley, Ratings: 4.0
  - Título: The Dead Zone, Autor: Stephen King, Ratings: 4.1764708


##### Agregación para contar documentos por autor

In [20]:
query = {
    "size": 0,
    "aggs": {
        "authors": {
            "terms": {
                "field": "author_name.keyword",
                "size": 10
            }
        }
    }
}

res = client.search(index='my_index', body=query)

print("Número de libros por autor:")
for bucket in res['aggregations']['authors']['buckets']:
    print(f"  - Autor: {bucket['key']}, Número de libros: {bucket['doc_count']}")

Número de libros por autor:
  - Autor: R. L. Stine, Número de libros: 204
  - Autor: Stephen King, Número de libros: 75
  - Autor: Dean Koontz, Número de libros: 37
  - Autor: Christopher Pike, Número de libros: 36
  - Autor: H.P. Lovecraft, Número de libros: 28
  - Autor: Steve Hutchison, Número de libros: 26
  - Autor: V. C. Andrews, Número de libros: 25
  - Autor: Anne Rice, Número de libros: 23
  - Autor: Karl Edward Wagner, Número de libros: 20
  - Autor: Edgar Allan Poe, Número de libros: 15


##### Búsqueda por idiomas

In [21]:
languages = ["spa", "eng"]

query = {
    "query": {
        "bool": {
            "should": [{"match": {"language": lang}} for lang in languages]
        }
    }
}

res = client.search(index='my_index', body=query)
language_names = ', '.join(languages)

print(f"Libros disponibles en {language_names}:")
for hit in res['hits']['hits']:
    source = hit['_source']
    titulo = source.get('title', 'Título desconocido')
    autor = source.get('author_name', 'Autor desconocido')
    idiomas = source.get('language', [])
    print(f"  - Título: {titulo}, Autor: {autor}, Idiomas: {idiomas}")

Libros disponibles en spa, eng:
  - Título: Horrorstör, Autor: Grady Hendrix, Idiomas: ['spa']
  - Título: El horror econòmico, Autor: Viviane Forrester, Idiomas: ['spa']
  - Título: Pauline, la noche del horror, Autor: Fco. Xavier Ramírez S., Idiomas: ['spa']
  - Título: Ficciones del horror, Autor: Carlos Surghi, Idiomas: ['spa']
  - Título: Fe de horrores, Autor: Aurelio Arteta, Idiomas: ['spa']
  - Título: Leyendas de Horror, Autor: Guillermo Murray Prisant, Idiomas: ['spa']
  - Título: Horrores estridentes, Autor: Diego Lizarazo Arias, Idiomas: ['spa']
  - Título: El ángulo del horror, Autor: Cristina Fernández Cubas, Idiomas: ['spa']
  - Título: Spanish horror, Autor: Víctor Matellano García, Idiomas: ['spa']
  - Título: Horror Secreto, El, Autor: H.P. Lovecraft, Idiomas: ['spa']


##### Búsqueda por año de publicación

In [22]:
start_year = 1990
end_year = 2000

query = {
    "query": {
        "range": {
            "publish_year": {
                "gte": start_year,
                "lte": end_year
            }
        }
    }
}

res = client.search(index='my_index', body=query)

print(f"Libros publicados entre {start_year} y {end_year}:")
for hit in res['hits']['hits']:
    source = hit['_source']
    titulo = source.get('title', 'Título desconocido')
    autor = source.get('author_name', 'Autor desconocido')
    publish_year = source.get('publish_year', 'Año desconocido')
    print(f"  - Título: {titulo}, Autor: {autor}, Año de Publicación: {publish_year}")

Libros publicados entre 1990 y 2000:
  - Título: Bag of Bones, Autor: Stephen King, Año de Publicación: 1998.0
  - Título: Four Past Midnight, Autor: Stephen King, Año de Publicación: 1990.0
  - Título: Insomnia, Autor: Stephen King, Año de Publicación: 1994.0
  - Título: Northern Lights, Autor: Philip Pullman, Año de Publicación: 1995.0
  - Título: The Regulators, Autor: Stephen King, Año de Publicación: 1996.0
  - Título: Needful Things, Autor: Stephen King, Año de Publicación: 1991.0
  - Título: The Green Mile, Autor: Stephen King, Año de Publicación: 1996.0
  - Título: The Subtle Knife, Autor: Philip Pullman, Año de Publicación: 1997.0
  - Título: Tales of Horror and Mystery, Autor: Roald Dahl, Año de Publicación: 1993.0
  - Título: The best horror stories, Autor: Bloch, Robert, Año de Publicación: 1990.0


##### Agregación por número de páginas

In [23]:
num_examples = 10

query = {
    "size": 0,
    "aggs": {
        "page_ranges": {
            "range": {
                "field": "num_pages",
                "ranges": [
                    {"to": 100},
                    {"from": 100, "to": 300},
                    {"from": 300, "to": 500},
                    {"from": 500}
                ]
            },
            "aggs": {
                "sample_books": {
                    "top_hits": {
                        "size": num_examples,
                        "_source": ["title", "author_name"]
                    }
                }
            }
        }
    }
}

res = client.search(index='my_index', body=query)

print("Distribución de libros por número de páginas:")
for bucket in res['aggregations']['page_ranges']['buckets']:
    range_label = f"{bucket['from']} - {bucket['to']} páginas" if 'from' in bucket and 'to' in bucket else \
                  (f"Menos de {bucket['to']} páginas" if 'to' in bucket else f"Más de {bucket['from']} páginas")
    example_books = bucket['sample_books']['hits']['hits']
    print(f"{range_label}: {bucket['doc_count']} libros")
    if example_books:
        print(f"Ejemplos de libros en este rango:")
        for example in example_books:
            example_source = example['_source']
            example_title = example_source.get('title', 'Título desconocido')
            example_author = example_source.get('author_name', 'Autor desconocido')
            print(f"  - {example_title} por {example_author}")
    else:
        print("  No hay ejemplos disponibles")

Distribución de libros por número de páginas:
Menos de 100.0 páginas: 180 libros
Ejemplos de libros en este rango:
  - Daisy Miller por Henry James
  - The Haunted House por Charles Dickens
  - The Willows por Algernon Blackwood
  - The Great God Pan por Arthur Machen
  - The Parasite por Arthur Conan Doyle
  - The Polar Express Gift Set por Chris Van Allsburg
  - Bunnicula por Deborah Howe
  - The Haunted and the Haunters, Or, The House and the Brain por Edward Bulwer Lytton, Baron Lytton
  - The Fall of the House of Usher por Edgar Allan Poe
  - Rikki-Tikki-Tavi por Rudyard Kipling
100.0 - 300.0 páginas: 856 libros
Ejemplos de libros en este rango:
  - Frankenstein or The Modern Prometheus por Mary Wollstonecraft Shelley
  - A Christmas Carol por Charles Dickens
  - The Picture of Dorian Gray por Oscar Wilde
  - Kalender 2021 por Paula Paulsen
  - Northanger Abbey por Jane Austen
  - The Strange Case of Dr. Jekyll and Mr. Hyde por Robert Louis Stevenson
  - Flatland por Edwin Abbott 

##### Búsqueda por nombre del autor

In [24]:
author_name = "Stephen King"

query = {
    "query": {
        "match": {
            "author_name": author_name
        }
    }
}

res = client.search(index='my_index', body=query)

print(f"Libros escritos por {author_name}:")
for hit in res['hits']['hits']:
    source = hit['_source']
    titulo = source.get('title', 'Título desconocido')
    publish_year = source.get('publish_year', 'Año desconocido')
    ratings = source.get('ratings', 'Ratings desconocidos')
    print(f"  - Título: {titulo}, Año de Publicación: {publish_year}, Ratings: {ratings}")

Libros escritos por Stephen King:
  - Título: It, Año de Publicación: 1986.0, Ratings: 4.109005
  - Título: Works (Carrie / Night Shift / 'Salem's Lot / Shining), Año de Publicación: 1981.0, Ratings: 4.5555553
  - Título: The Stand, Año de Publicación: 1978.0, Ratings: 4.285714
  - Título: Black House, Año de Publicación: 2001.0, Ratings: 3.5625
  - Título: The Bachman Books, Año de Publicación: 1985.0, Ratings: 3.7435896
  - Título: Skeleton Crew, Año de Publicación: 1985.0, Ratings: 4.347826
  - Título: Different Seasons, Año de Publicación: 1982.0, Ratings: 4.5882354
  - Título: Bag of Bones, Año de Publicación: 1998.0, Ratings: 3.9629629
  - Título: Salem’s Lot, Año de Publicación: 1975.0, Ratings: 4.0882354
  - Título: Four Past Midnight, Año de Publicación: 1990.0, Ratings: 3.9473684


##### Búsqueda por palabra clave en el tema (subject)

In [25]:
keyword = "Stories"

query = {
    "query": {
        "match": {
            "subject": keyword
        }
    }
}

res = client.search(index='my_index', body=query)

print(f"Documentos con la palabra '{keyword}' en el sujeto:")
for hit in res['hits']['hits']:
    source = hit['_source']
    title = source.get('title', 'Título no disponible')
    author_name = source.get('author_name', 'Autor no disponible')
    print(f"  - Título: {title}, Autor: {author_name}")

Documentos con la palabra 'Stories' en el sujeto:
  - Título: The Complete Horowitz Horror, Autor: Anthony Horowitz
  - Título: Horowitz Horror Stories You'll Wish You Never Read, Autor: Anthony Horowitz
  - Título: Beast, Autor: Scott Ingram
  - Título: The Accident, Autor: Diane Hoh
  - Título: Give Yourself Goosebumps - Escape From the Carnival of Horrors, Autor: R. L. Stine
  - Título: Collected Ghost Stories, Autor: Montague Rhodes James
  - Título: Choose Your Own Adventure - Horror House, Autor: Edward Packard
  - Título: Bury Me Deep, Autor: Christopher Pike
  - Título: Tales to Give You Goosebumps, Autor: R. L. Stine
  - Título: Horrors, horrors, horrors, Autor: Helen Hoke


In [26]:
if client.indices.exists(index='my_index'):
    client.indices.delete(index='my_index')
    print(f"El índice my_index ha sido eliminado.")
else:
    print(f"El índice my_index no existe.")

El índice my_index ha sido eliminado.
